In [1]:
from comet_ml import Experiment
from comet_ml import API

In [2]:
import numpy as np 
import shap
import pickle
import json
import time
import json
import pandas as pd
import torch
import math
import os
import torch.nn as nn
import torch.nn.functional as F
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, roc_curve, auc
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import sys
from binary_classifier import *
sys.path.append('../ift6758/visualizations/')
#from performance_vis import *

In [3]:
df = pd.read_csv('final_df.csv')

In [4]:
api= API(api_key='PeZE4vLrXZR7BuQTNrIYRlRF9')

In [5]:
experiment_rf = api.get("kleitoun/random-forest/question6")
experiment_lr_d= api.get("kleitoun/logistic-regression-models/lrd")
experiment_lr_a= api.get("kleitoun/logistic-regression-models/lra")
experiment_lr_da= api.get("kleitoun/logistic-regression-models/lrda")
experiment_xgb = api.get("kleitoun/milestone-2/xgboost_tuning")
experiment_nn = api.get("kleitoun/neural-net/chief_moth_9230")
experiment_rf.download_model('randomforest',output_path="./",expand=True)
experiment_lr_d.download_model('LR_D',output_path="./",expand=True)
experiment_lr_a.download_model('LR_A',output_path="./",expand=True)
experiment_lr_da.download_model('LR_DA',output_path="./",expand=True)
experiment_xgb.download_model('XGBoost_hmtuning_model_v2',output_path="./",expand=True)
experiment_nn.download_model('nn_final',output_path="./",expand=True)

COMET INFO: Downloading experiment model 'randomforest' ...
COMET INFO: Unzipping model to './' ...
COMET INFO: done!
COMET INFO: Downloading experiment model 'LR_D' ...
COMET INFO: Unzipping model to './' ...
COMET INFO: done!
COMET INFO: Downloading experiment model 'LR_A' ...
COMET INFO: Unzipping model to './' ...
COMET INFO: done!
COMET INFO: Downloading experiment model 'LR_DA' ...
COMET INFO: Unzipping model to './' ...
COMET INFO: done!
COMET INFO: Downloading experiment model 'XGBoost_hmtuning_model_v2' ...
COMET INFO: Unzipping model to './' ...
COMET INFO: done!
COMET INFO: Downloading experiment model 'nn_final' ...
COMET INFO: Unzipping model to './' ...
COMET INFO: done!


In [6]:
df_test = df[df['season'] == 20192020]
df_test_R = df_test[df_test['season_type'] == 'R']
df_test_P = df_test[df_test['season_type'] == 'P']

In [7]:
rf = pickle.load(open("random_forest", 'rb'))
#xgb = pickle.load(open("XGBoost_hmtuning_model_v2.pickle",'rb'))
log1 = pickle.load(open("LogisticRegression_distance", 'rb'))
log2 = pickle.load(open("LogisticRegression_angle", 'rb'))
log3 = pickle.load(open("LogisticRegression_distance+angle", 'rb'))

## Create pytorch datasets, load pytorch model

In [8]:
old_feat = ['game_id', 'Unnamed: 0',  'event_id', 'home_team',
       'away_team', 'home_team_side_1st_period', 'attacking_team',
       'attacking_player', 'goalie', 'period', 'period_time',
       'shot_ind', 'x_coordinates', 'y_coordinates', 'shot_type', 'empty_net',
       'strength', 'gwg', 'previous_event_x_coordinates',
       'previous_event_y_coordinates', 'previous_event_period_time',
       'previous_event_type', 'previous_event_period', 'previous_attacking_team',
       'attacking_team_side']

df_feat = df_test.iloc[:,:60].columns.tolist()

#features extracted in question 4
new_feat = [feat for feat in df_feat if feat not in old_feat]

df_test_nn = df_test[new_feat]

cols_to_normalize = list(df_test_nn.columns)
cols_to_normalize.remove('season')
cols_to_normalize.remove('goal_ind')
cols_to_normalize.remove('season_type')

for col in cols_to_normalize:
    df_test_nn[col] = (df_test_nn[col] - min(df_test_nn[col]))/(max(df_test_nn[col])-min(df_test_nn[col]))

df_test_nn_R = df_test_nn[df_test_nn['season_type'] == 'R']
df_test_nn_P = df_test_nn[df_test_nn['season_type'] == 'P']

X_nn_R = df_test_nn_R.drop(columns = ['season', 'goal_ind', 'season_type'])
Y_nn_R = df_test_nn_R['goal_ind']
X_nn_R = torch.tensor(X_nn_R.to_numpy()).float()
Y_nn_R = torch.tensor(Y_nn_R.to_numpy())

X_nn_P = df_test_nn_P.drop(columns = ['season', 'goal_ind', 'season_type'])
Y_nn_P = df_test_nn_P['goal_ind']
X_nn_P = torch.tensor(X_nn_P.to_numpy()).float()
Y_nn_P = torch.tensor(Y_nn_P.to_numpy())


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
nn_model = binary_classifier()
nn_model.load_state_dict(torch.load('q6_neural_net_final'))

<All keys matched successfully>

## Features for RF

In [10]:
old_feat = ['game_id', 'season', 'season_type', 'event_id', 'home_team',
       'away_team', 'home_team_side_1st_period', 'attacking_team',
       'attacking_player', 'goalie', 'period', 'period_time', 'goal_ind',
       'shot_ind', 'x_coordinates', 'y_coordinates', 'shot_type', 'empty_net',
       'strength', 'gwg', 'previous_event_x_coordinates',
       'previous_event_y_coordinates', 'previous_event_period_time',
       'previous_event_type', 'previous_event_period', 'previous_attacking_team',
       'attacking_team_side']

df_feat = df.iloc[:,:60].columns.tolist()

RF_feat = [feat for feat in df_feat if feat not in old_feat]

## Features for XGBoost

In [11]:
feat_xgb = ['game_seconds', 'period', 'x_coordinates', 'y_coordinates',
       'distance_from_net', 'angle_from_net', 'previous_event_type',
       'previous_event_x_coordinates', 'previous_event_y_coordinates',
       'time_since_last_event', 'distance_from_last_event', 'rebound',
       'change_in_angle', 'speed', 'time_since_powerplay_started', '5v5',
       '4v4', '3v3', '5v4', '5v3', '4v3', '4v5', '3v5', '3v4',
       'shot_type_Backhand', 'shot_type_Deflected', 'shot_type_Slap Shot',
       'shot_type_Snap Shot', 'shot_type_Tip-In', 'shot_type_Wrap-around',
       'shot_type_Wrist Shot']

def preprocess(df):
    le = LabelEncoder()
    df["previous_event_type"] = le.fit_transform(df["previous_event_type"])
    
    return df

le_name_mapping = {'Blocked Shot': 0, 'Faceoff': 1, 'Game Official': 2, 'Giveaway': 3, 'Goal': 4, 'Hit': 5, 'Missed Shot': 6, 'Official Challenge': 7, 'Penalty': 8, 'Period End': 9, 'Period Official': 10, 'Period Ready': 11, 'Period Start': 12, 'Shootout Complete': 13, 'Shot': 14, 'Stoppage': 15, 'Takeaway': 16}

## Assign the proper features to each model

In [12]:
X_rf = df_test_R[RF_feat]
X_rf_P = df_test_P[RF_feat]
X_xgb = preprocess(df_test_R[feat_xgb])
X_xgb["previous_event_type"] = X_xgb["previous_event_type"].replace(le_name_mapping)
X_xgb_P = preprocess(df_test_P[feat_xgb])
X_xgb_P["previous_event_type"] = X_xgb_P["previous_event_type"].replace(le_name_mapping)
X_lr_D = df_test_R['distance_from_net'].values.reshape(-1,1)
X_lr_D_P = df_test_P['distance_from_net'].values.reshape(-1,1)
X_lr_A = df_test_R['angle_from_net'].abs().values.reshape(-1,1)
X_lr_A_P = df_test_P['angle_from_net'].abs().values.reshape(-1,1)
X_lr_DA = df_test_R[['distance_from_net','angle_from_net']]
X_lr_DA_P = df_test_P[['distance_from_net','angle_from_net']]
y = df_test_R['goal_ind']
y_P = df_test_P['goal_ind']


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Prediction for regular season 

In [13]:
preds_rf = rf.predict(X_rf)
#preds_xgb = xgb.predict(X_xgb)
preds_lr_d = log1.predict(X_lr_D)
preds_lr_a = log2.predict(X_lr_A)
preds_lr_da = log3.predict(X_lr_DA)

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Unnamed: 0
Feature names seen at fit time, yet now missing:
- shot_type_Wrist Shot

X has feature names, but LogisticRegression was fitted without feature names


## Accuracy and ROC for logistic regression models

In [14]:
accuracy_lrd = accuracy_score(y,preds_lr_d)
roc_auc_lrd =  roc_auc_score(y,preds_lr_d)
accuracy_lra = accuracy_score(y,preds_lr_a)
roc_auc_lra =  roc_auc_score(y,preds_lr_a)
accuracy_lrda = accuracy_score(y,preds_lr_da)
roc_auc_lrda =  roc_auc_score(y,preds_lr_da)

In [15]:
nn_predicted_probs_P = nn_model.predict_probs(X_nn_P)[:,1].detach().numpy()
nn_predicted_probs_R = nn_model.predict_probs(X_nn_R)[:,1].detach().numpy()

In [16]:
roc_auc_score(y, nn_predicted_probs_R)

0.7479192884480441

In [17]:
roc_auc_score(y_P, nn_predicted_probs_P)

0.6673380992358794

## Accuracy and ROC for Random Forest

In [13]:
accuracy_rf = accuracy_score(y,preds_rf)
roc_auc_rf =  roc_auc_score(y,preds_rf)
report_rf = classification_report(y, preds_rf, output_dict=True)

## Accuracy and ROC for XGBoost

In [14]:
accuracy_xgb = accuracy_score(y,preds_xgb)
roc_auc_xgb =  roc_auc_score(y,preds_xgb)
report_xgb = classification_report(y, preds_xgb, output_dict=True)

## Plots for regular season

## Predictions for playoffs

In [15]:
preds_rf_p = rf.predict(X_rf_P)
preds_xgb_p = xgb.predict(X_xgb_P)
preds_lr_d_p = log1.predict(X_lr_D_P)
preds_lr_a_p = log2.predict(X_lr_A_P)
preds_lr_da_p = log3.predict(X_lr_DA_P)

X has feature names, but LogisticRegression was fitted without feature names


## Accuracy and ROC for logistic regression models - playoffs

In [16]:
accuracy_lrd_p = accuracy_score(y_P,preds_lr_d_p)
roc_auc_lrd_p =  roc_auc_score(y_P,preds_lr_d_p)
accuracy_lra_p = accuracy_score(y_P,preds_lr_a_p)
roc_auc_lra_p =  roc_auc_score(y_P,preds_lr_a_p)
accuracy_lrda_p = accuracy_score(y_P,preds_lr_da_p)
roc_auc_lrda_p =  roc_auc_score(y_P,preds_lr_da_p)

## Accuracy and ROC for Random Forest - playoffs

In [17]:
accuracy_rf_p = accuracy_score(y_P,preds_rf_p)
roc_auc_rf_p =  roc_auc_score(y_P,preds_rf_p)
report_rf_p = classification_report(y_P, preds_rf_p, output_dict=True)

## Accuracy and ROC for XGBoost - playoffs

In [18]:
accuracy_xgb_p = accuracy_score(y_P,preds_xgb_p)
roc_auc_xgb_p =  roc_auc_score(y_P,preds_xgb_p)
report_xgb_p = classification_report(y_P, preds_xgb_p, output_dict=True)

## Plots for playoffs 